In [31]:
# futures_grid_bot_sim.py
# Full simulation pipeline — uses live data from Binance via ccxt, but DOES NOT place real trades.
# It will open LONG grids when model predicts uptrend and SHORT grids when model predicts downtrend.
# No leverage is used (placeholder comments show where to add leverage later).

import ccxt
import torch
import torch.nn as nn
import joblib
import numpy as np
import pandas as pd
import ta
import talib
import uuid
import time
from datetime import datetime, timedelta, timezone


In [32]:
# -------------------------
# CONFIG
# -------------------------
symbol = "BTC/USDT"
market_type = "futures"        # we fetch futures-market data (via ccxt options)
model_timeframe = "15m"
price_timeframe = "1s"
limit = 300
window_size = 10

FEATURES = [
    'rsi', 'ema12', 'ema26', 'macd', 'signal', 'histogram',
    'dema9', 'sma3', 'tsi', '%k', '%d',
    'atr', 'obv', 'adx', 'cci',
    'return_1', 'roll_mean_5', 'roll_std_5'
]

SAVED_MODEL = "greg_tech_8.pth"
SCALER_FILE = "scaler_15m.pkl"

min_tp_pct = 0.01
atr_period = 14
atr_multiplier = 0.1
position_fraction = 0.05
reduced_fraction = 0.5
grid_levels = 16
grid_spacing_atr_factor = 0.1
max_total_exposure = 0.7

prediction_interval = 15 * 60
check_interval_seconds = 1
simulate_only = True  # simulation mode — no real trades
hold_factor = 0.2

# Forced cutoff parameters for frozen grids
frozen_timeout_seconds = 60 * 60
frozen_max_drawdown_pct = 0.2

# device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [33]:
# -------------------------
# MODEL (your LSTM)
# -------------------------
class CryptoLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(CryptoLSTM, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        out, (hn, cn) = self.lstm(x)
        out = out[:, -1, :]
        out = self.fc(out)
        return out

input_dim = len(FEATURES)
hidden_dim = 128
num_layers = 4
output_dim = 3

model = CryptoLSTM(input_dim, hidden_dim, num_layers, output_dim).to(device)
model.load_state_dict(torch.load(SAVED_MODEL, map_location=device))
model.eval()
scaler = joblib.load(SCALER_FILE)

/home/gregory/Trading_bot/venv/lib/python3.12/site-packages/sklearn/base.py:442: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.6.1 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [34]:
# -------------------------
# CCXT EXCHANGE SETUP (Futures data)
# -------------------------
exchange = ccxt.binance({
    'apiKey': 'eOUQOwsdCjiNMUDbNsrTV0TUPPF7rAiEXiWtk5CMEGClviPoqBqFhm1KFddUtyoE',
    'secret': 'rpPI79M1aoh8sc7oeZJTj8Fox3ppcBPhtBvWXtTBCWW65q4Ys23tBxo5MZ8tgIZv',
    'options': {
        'defaultType': 'future',  # Use Futures endpoints
    },
    'urls': {
        'api': {
            'fapiPublic': 'https://testnet.binancefuture.com/fapi/v1',
            'fapiPrivate': 'https://testnet.binancefuture.com/fapi/v1',
        }
    }
})
exchange.enable_demo_trading(True)  # Enable demo trading mode


In [35]:
# Helper: fetch minimums / step sizes for rounding
MARKET_INFO = {}
def load_market_info():
    global MARKET_INFO
    markets = exchange.load_markets()
    if symbol in markets:
        info = markets[symbol]
        MARKET_INFO['precision'] = info.get('precision', {})
        MARKET_INFO['limits'] = info.get('limits', {})
    else:
        MARKET_INFO['precision'] = {}
        MARKET_INFO['limits'] = {}


In [36]:
# -------------------------
# INDICATORS (same compute_indicators but lowercase columns)
# -------------------------
def compute_indicators(df):
    df = df.copy()
    df.columns = [c.lower() for c in df.columns]

    out = {}
    # Core
    out["RSI"] = ta.momentum.RSIIndicator(df["close"], window=14).rsi()
    out["EMA12"] = df["close"].ewm(span=12, adjust=False).mean()
    out["EMA26"] = df["close"].ewm(span=26, adjust=False).mean()
    out["MACD"] = out["EMA12"] - out["EMA26"]
    out["Signal"] = out["MACD"].ewm(span=9, adjust=False).mean()
    out["Histogram"] = out["MACD"] - out["Signal"]
    # DEMA
    try:
        out["DEMA9"] = talib.DEMA(df["close"].values, timeperiod=9)
    except Exception:
        out["DEMA9"] = df["close"].ewm(span=9, adjust=False).mean()
    # SMA
    out["SMA3"] = ta.trend.sma_indicator(df["close"], window=3)
    # TSI
    def compute_tsi(close, r1=25, r2=13):
        delta = close.diff()
        ema1 = delta.ewm(span=r1, adjust=False).mean()
        ema2 = ema1.ewm(span=r2, adjust=False).mean()
        abs_delta = delta.abs()
        abs_ema1 = abs_delta.ewm(span=r1, adjust=False).mean()
        abs_ema2 = abs_ema1.ewm(span=r2, adjust=False).mean()
        tsi = 100 * (ema2 / (abs_ema2.replace(0, np.nan)))
        return tsi.fillna(0)
    out["TSI"] = compute_tsi(df["close"])
    # Stochastic
    period, smooth_k, smooth_d = 14, 3, 3
    lowest_low = df["low"].rolling(period).min()
    highest_high = df["high"].rolling(period).max()
    out["%K"] = 100 * (df["close"] - lowest_low) / (highest_high - lowest_low + 1e-8)
    out["%K"] = out["%K"].rolling(smooth_k).mean()
    out["%D"] = out["%K"].rolling(smooth_d).mean()
    # Advanced
    out["ATR"] = ta.volatility.AverageTrueRange(df["high"], df["low"], df["close"], window=14).average_true_range()
    out["OBV"] = ta.volume.OnBalanceVolumeIndicator(df["close"], df["volume"]).on_balance_volume()
    out["ADX"] = ta.trend.ADXIndicator(df["high"], df["low"], df["close"], window=14).adx()
    out["CCI"] = ta.trend.cci(df["high"], df["low"], df["close"], window=20)
    # Price derived
    out["Return_1"] = df["close"].pct_change()
    out["Roll_Mean_5"] = df["close"].rolling(5).mean()
    out["Roll_Std_5"] = df["close"].rolling(5).std()

    feat_df = pd.DataFrame(out, index=df.index)
    feat_df = feat_df.dropna().reset_index(drop=True)
    feat_df.columns = [c.lower() for c in feat_df.columns]
    return feat_df

In [37]:
# -------------------------
# FETCH DATA
# -------------------------
def fetch_latest_ohlcv(symbol, timeframe, limit):
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe=timeframe, limit=limit)
    df = pd.DataFrame(ohlcv, columns=['timestamp','open','high','low','close','volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)
    df.columns = [c.lower() for c in df.columns]
    return df

def get_current_atr():
    df = fetch_latest_ohlcv(symbol, model_timeframe, atr_period + 5)
    df['prev_close'] = df['close'].shift(1)
    df['TR'] = df.apply(lambda r: max(r['high'] - r['low'],
                                      abs(r['high'] - r['prev_close']),
                                      abs(r['low'] - r['prev_close'])), axis=1)
    df['ATR'] = df['TR'].rolling(atr_period).mean()
    return float(df['ATR'].iloc[-1])


In [38]:
# -------------------------
# CCXT EXCHANGE SETUP
# -------------------------
exchange_2 = ccxt.binance({
    'enableRateLimit': True,
})

In [39]:
def fetch_latest_ohlcv_2(symbol, timeframe, limit):
    ohlcv = exchange_2.fetch_ohlcv(symbol, timeframe=timeframe, limit=limit)
    df = pd.DataFrame(ohlcv, columns=['timestamp','open','high','low','close','volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)
    df.columns = [c.lower() for c in df.columns]
    return df

In [40]:
# -------------------------
# Prediction
# -------------------------
def predict_label_from_window(window_features):
    # window_features: numpy array shape (window_size, features)
    flat = scaler.transform(window_features)
    x = torch.tensor(flat, dtype=torch.float32).unsqueeze(0).to(device)
    
    # No manual biasing — purely model-based prediction
    with torch.no_grad():
        logits = model(x)
        probs = torch.softmax(logits, dim=1).cpu().numpy()[0]
        pred = int(np.argmax(probs))
    
    return pred  # 0 = down, 1 = hold, 2 = up


In [41]:
# -------------------------
# Trading state & helpers
# -------------------------
from decimal import Decimal, ROUND_DOWN

balance = 100.0
open_positions = []   # each: {id, side: 'long'/'short', entry, qty, notional(size_usd), grid_id, created_at, tp_price, sl_price, last_price}
open_orders = []      # each: {id, grid_id, side, price, size_usd, status, ccxt_id, tp, sl}
grids = {}            # grid metadata
blocked_zones = []   

def usd_to_qty(usd, price):
    return usd / price

def round_amount_qty(qty, symbol_local=None):
    """
    Round quantity according to market precision and ensure it's >= exchange min amount.
    Returns a float suitable for order placement.
    """
    try:
        symbol_send = symbol_local if symbol_local else symbol
        market = exchange.market(symbol_send)
        
        # Get amount precision
        prec = int(market.get('precision', {}).get('amount', 6))
        
        # Get min amount
        min_amount = float(market.get('limits', {}).get('amount', {}).get('min', 10**(-prec)))
        
        # Round down with Decimal
        quant = Decimal(str(qty)).quantize(Decimal('1e-{}'.format(prec)), rounding=ROUND_DOWN)
        qty_rounded = max(Decimal(str(min_amount)), quant)
        
        # Safety: check if rounded qty is valid
        if qty_rounded < Decimal(str(min_amount)):
            print(f"❌ Qty {qty_rounded} below min {min_amount}, skipping order.")
            return 0.0
        
        return float(qty_rounded)
    
    except Exception as e:
        print(f"[ROUND_AMOUNT_QTY ERROR] {e}")
        # fallback: simple round with 6 decimals
        return float(round(qty, 6))

def round_price(price):
    prec = MARKET_INFO.get('precision', {}).get('price')
    if prec is None:
        return price
    prec = int(prec)
    return float(round(price, prec))


In [42]:
HEDGE_MODE = None  # None => try to auto-detect, True/False => force

def detect_hedge_mode():
    """
    Try to detect whether the futures account is in hedge (dual-side) mode.
    If detection fails, return False (safe default).
    """
    global HEDGE_MODE
    if HEDGE_MODE is not None:
        return bool(HEDGE_MODE)
    try:
        # CCXT sometimes exposes this raw endpoint; try a couple of possibilities
        # If these calls fail, we fallback to False (one-way mode).
        try:
            # newer ccxt: fapiPrivate_get_positionSide_dual or fapiPrivateGetPositionSideDual
            # try both spellings
            r = getattr(exchange, 'fapiPrivate_get_positionSide_dual', None)
            if callable(r):
                resp = r()
                val = resp.get('dualSidePosition') if isinstance(resp, dict) else None
                if val is not None:
                    HEDGE_MODE = bool(val)
                    return HEDGE_MODE
        except Exception:
            pass

        # fallback try fetch_positions and look for 'positionSide' in info
        try:
            positions = exchange.fetch_positions()
            for p in positions:
                info = p.get('info') if isinstance(p, dict) else None
                if info and ('positionSide' in info or 'positionSide' in p):
                    HEDGE_MODE = True
                    return True
        except Exception:
            pass

    except Exception:
        pass

    # default safe: assume one-way mode
    HEDGE_MODE = False
    return False

In [43]:
# -------------------------
# FUTURES order helpers (simulated)
# -------------------------
    
def place_market_order_ccxt(side, amount, pos_side=None, reduce_only=False, symbol_local=None):
    """
    Unified market order helper for futures (demo/live) in hedge mode.
        side: 'buy' or 'sell' (action)
        amount: already-calculated asset amount (float)
        pos_side: 'LONG' or 'SHORT' (required in hedge mode)
        reduce_only: boolean; include reduceOnly param only when True
        symbol_local: optional symbol string (e.g., "BTC/USDT") to use for precision rounding
    Returns a dict-like result (never None) with keys including 'id','amount','status','error' on failure.
    """
    symbol_send = symbol_local if symbol_local else symbol  # use global symbol fallback
    amount = round_amount_qty(amount, symbol_send)
    if amount <= 0:
        return {'id': None, 'amount': 0, 'status': 'failed', 'error': 'amount<=0 after rounding'}

    params = {}
    # Always include positionSide in hedge mode
    if pos_side:
        params['positionSide'] = pos_side

    # Only add reduceOnly if explicitly requested
    if reduce_only:
        params['reduceOnly'] = True

    try:
        # Market order: price=None
        order = exchange.create_order(symbol_send, 'market', side, amount, None, params)
        # Normalize response
        return {
            'id': order.get('id') if isinstance(order, dict) else getattr(order, 'id', None),
            'average': float(order.get('average', 0)) if isinstance(order, dict) else float(getattr(order, 'average', 0)),
            'amount': float(order.get('filled', order.get('amount', amount))) if isinstance(order, dict) else amount,
            'status': order.get('status', 'closed') if isinstance(order, dict) else 'closed',
            'raw': order
        }
    except Exception as e:
        print(f"❌ Order placement failed: {e}")
        return {'id': None, 'amount': 0, 'status': 'failed', 'error': str(e)}


    
def fetch_order_ccxt(ccxt_order_id):
    try:
        return exchange.fetch_order(ccxt_order_id, symbol)
    except Exception as e:
        print("[CCXT FETCH ORDER ERROR]", e)
        return None
    
            
        
def cancel_order_ccxt(ccxt_order_id):
    try:
        exchange.cancel_order(ccxt_order_id, symbol)
        return True
    except Exception as e:
        print("[CCXT CANCEL ERROR]", e)
        return False        
        
def market_close_position_ccxt(pos):
    """
    Close open position (simulated) — return a dict containing pnl if simulated.
    """   
    try:
        # For real futures: place market order reduceOnly = True and proper positionSide
        amount = round_amount_qty(pos['qty'])
        close_side = 'sell' if pos['side'] == 'long' else 'buy'
        params = {'reduceOnly': True}
        order = exchange.create_order(symbol, 'market', close_side, amount, None, params)
        return order
    except Exception as e:
        print("[CCXT MARKET CLOSE ERROR]", e)
        return None        

In [44]:
user_grid_spacing = 50.0  # fixed spacing in USD between grid levels

def recreate_grid_order(price, side):
    """
    Recreates a grid order after TP/SL is hit,
    using the same spacing logic as place_grid().
    """
    global open_orders, grids

    # Find the matching grid for this price
    grid_id = None
    for gid, meta in grids.items():
        for oid in meta.get('orders', []):
            for o in open_orders:
                if abs(o['price'] - price) < 1e-8 and o['grid_id'] == gid:
                    grid_id = gid
                    break
    if grid_id is None and len(grids) > 0:
        grid_id = list(grids.keys())[0]

    # Use the same user-defined spacing, not ATR
    spacing = user_grid_spacing
    size_usd = max(20.0, balance * position_fraction)

    # Define TP and SL based on direction (side)
    if side == 'long':
        tp = price + spacing
        sl = price - 0.5 * spacing
        order_side = 'buy'
    else:  # short
        tp = price - spacing
        sl = price + 0.5 * spacing
        order_side = 'sell'

    # Create the order object
    order = {
        'id': str(uuid.uuid4())[:10],
        'grid_id': grid_id,
        'side': order_side,
        'price': float(price),
        'size_usd': float(size_usd),
        'status': 'open',
        'tp': float(tp),
        'sl': float(sl)
    }

    open_orders.append(order)
    if grid_id:
        grids[grid_id]['orders'].append(order['id'])

    print(f"[GRID REUSED] {order_side.upper()} order re-created at {price:.2f} (tp={tp:.2f}, sl={sl:.2f})")


In [45]:
# -------------------------
# Place grid (supports 'up' -> long grid, 'down' -> short grid)
# -------------------------
user_grid_spacing = 100.0  # fixed spacing in USD between grid levels    
def place_grid(grid_direction, center_price, mode='trend', fraction=position_fraction):
    """
    Symmetric ladder grid system:
    - For 'up' we create buy levels (long entries)
    - For 'down' we create sell levels (short entries)
    """
    gid = str(uuid.uuid4())[:8]
    grids[gid] = {
        'id': gid,
        'direction': grid_direction,
        'active': True,
        'created_at': datetime.utcnow(),
        'orders': [],
        'frozen': False
    }

    size_usd = max(20.0, balance * fraction)

    # Generate grid prices using user-defined spacing
    half = grid_levels // 2
    grid_prices = [center_price + i * user_grid_spacing for i in range(-half, half + 1)]
    grid_prices = sorted(set(grid_prices))

    # for down (short grid) we use 'sell' entries, for up we use 'buy'
    order_side = 'sell' if grid_direction == 'down' else 'buy'

    for i, level_price in enumerate(grid_prices):
    # Handle Take-Profit (TP)
        if grid_direction == 'up':
            # TP should be the next higher grid (unless it's the last one)
            if i + 1 < len(grid_prices):
                tp_price = grid_prices[i + 1]
            else:
                tp_price = level_price * (1 + min_tp_pct)
            # SL slightly below the current level
            sl_price = level_price - 0.5 * user_grid_spacing

        else:  # grid_direction == 'down'
            # TP should be the next lower grid (unless it's the first one)
            if i > 0:
                tp_price = grid_prices[i - 1]
            else:
                tp_price = level_price * (1 - min_tp_pct)
            # SL slightly above the current level
            sl_price = level_price + 0.5 * user_grid_spacing    
        """try:
            #resp = place_limit_order_ccxt(order_side, level_price, size_usd)
            #ccxt_id = resp.get('id')
        except Exception:
            #ccxt_id = None"""

        oid = str(uuid.uuid4())[:10]
        order = {
            'id': oid,
            'grid_id': gid,
            'side': order_side,
            'price': float(level_price),
            'size_usd': float(size_usd),
            'status': 'open',
            'tp': float(tp_price),
            'sl': float(sl_price)
        }
        open_orders.append(order)
        grids[gid]['orders'].append(oid)

    print(f"[GRID PLACED] id={gid}, dir={grid_direction}, mode={mode}, levels={len(grid_prices)}")
    print("------------------------------------------------------------")
    print(f"[DEBUG] Grid Details for {gid}:")
    for i, level_price in enumerate(grid_prices):
        # Handle Take-Profit (TP)
        if grid_direction == 'up':
            # TP should be the next higher grid (unless it's the last one)
            if i + 1 < len(grid_prices):
                tp_price = grid_prices[i + 1]
            else:
                tp_price = level_price * (1 + min_tp_pct)
            # SL slightly below the current level
            sl_price = level_price - 0.5 * user_grid_spacing

        else:  # grid_direction == 'down'
            # TP should be the next lower grid (unless it's the first one)
            if i > 0:
                tp_price = grid_prices[i - 1]
            else:
                tp_price = level_price * (1 - min_tp_pct)
            # SL slightly above the current level
            sl_price = level_price + 0.5 * user_grid_spacing
        print(f"Grid {i+1}/{len(grid_prices)} | Price: {level_price:.2f} | TP: {tp_price:.2f} with difference = {tp_price - level_price} | SL: {sl_price:.2f} with difference = {sl_price - level_price}")
    print("------------------------------------------------------------")
    return gid

def freeze_grid(gid):
    if gid in grids:
        grids[gid]['frozen'] = True
        grids[gid]['active'] = False
        grids[gid]['frozen_at'] = datetime.now(timezone.utc)
        print(f"[GRID FROZEN] id={gid}")

def cancel_grid_orders_ccxt(gid):
    global open_orders
    to_cancel = [o for o in open_orders if o['grid_id'] == gid]
    for o in to_cancel:
        id = o.get('id')
        if id:
            open_orders.remove(o)
    print(f"[GRID CANCELLED] id={gid}, canceled_orders={len(to_cancel)}")
    grids.pop(gid, None)

def get_open_orders_for_grid(gid):
    return [o for o in open_orders if o['grid_id'] == gid and o['status'] == 'open']

def total_open_exposure():
    pos_exposure = sum([p['notional'] for p in open_positions])
    orders_exposure = sum([o['size_usd'] for o in open_orders if o['status'] == 'open'])
    return pos_exposure + orders_exposure

def has_position_from_grid(gid):
    return any(p.get('grid_id') == gid and p.get('active', True) for p in open_positions)

def get_next_higher_price(entry_price):
    buy_prices = sorted([o['price'] for o in open_orders if o['status']=='open' and o['side']=='buy'])
    for p in buy_prices:
        if p > entry_price:
            return p
    return entry_price * (1 + min_tp_pct)

In [46]:
# -------------------------
# Simulation: fill open limit orders when price touches them
# -------------------------
def try_fill_orders_sim_or_real(current_price):
    global balance
    filled = []
    if simulate_only:
        for o in list(open_orders):
            if o['status'] != 'open': continue
            grid_id = o['grid_id']
            grid_prices = sorted([oo['price'] for oo in open_orders if oo['grid_id'] == grid_id])
            i = grid_prices.index(o['price'])
            lower_bound = grid_prices[i - 1] if i > 0 else o['price'] * 0.995
            upper_bound = grid_prices[i + 1] if i + 1 < len(grid_prices) else o['price'] * 1.005
            next_prices = [p for p in grid_prices if p > o['price']]
            tp_price = next_prices[0] if next_prices else o['price'] * (1 + min_tp_pct)

            # zone occupation check
            zone_occupied = any(p['grid_id'] == grid_id and lower_bound < p['entry'] <= upper_bound for p in open_positions)
            if zone_occupied: continue

            # BUY fills -> LONG entries (price dips into zone)
            if o['side'] == 'buy' and lower_bound < current_price <= o['price']:
                size_usd = float(o['size_usd'])
                qty = usd_to_qty(size_usd, o['price'])
                balance -= size_usd   # reserve notional as margin (no leverage)
                pos = {
                    'id': str(uuid.uuid4())[:10],
                    'side': 'long',
                    'entry': float(o['price']),
                    'qty': qty,
                    'notional': size_usd,
                    'grid_id': o['grid_id'],
                    'created_at': datetime.now(timezone.utc),
                    'last_price': current_price,
                    'active': True,
                    'tp_price': float(o['tp']),
                    'sl_price': float(o['sl'])
                }
                open_positions.append(pos)
                o['status'] = 'filled'; filled.append(o)
                print(f"[SIM FILL BUY] order={o['id']} price={o['price']:.2f} qty={qty:.6f} notional={size_usd:.2f}")

            # SELL fills -> SHORT entries (price rises into sell level)
            elif o['side'] == 'sell' and current_price >= o['price']:
                size_usd = float(o['size_usd'])
                qty = usd_to_qty(size_usd, o['price'])
                balance -= size_usd   # reserve notional as margin (no leverage)
                # For shorts we store notional similarly and compute PnL on close
                pos = {
                    'id': str(uuid.uuid4())[:10],
                    'side': 'short',
                    'entry': float(o['price']),
                    'qty': qty,
                    'notional': size_usd,
                    'grid_id': o['grid_id'],
                    'created_at': datetime.now(timezone.utc),
                    'last_price': current_price,
                    'active': True,
                    'tp_price': float(o['tp']),
                    'sl_price': float(o['sl'])
                }
                open_positions.append(pos)
                o['status'] = 'filled'; filled.append(o)
                print(f"[SIM FILL SELL] order={o['id']} price={o['price']:.2f} qty={qty:.6f} notional={size_usd:.2f}")
    
    return filled


In [47]:
has_open_position_zones = set()

In [ ]:
def try_resolve_positions_by_tp_sl(current_price):
    global balance
    closed = []

    for pos in list(open_positions):
        amount = pos.get('qty', 0)
        if amount <= 0:
            print("[POSITION CLOSE SKIP] amount <= 0")
            continue  # nothing to close

        tp = pos.get('tp_price')
        sl = pos.get('sl_price')
        entry_price = pos['entry']
        grid_id = pos['grid_id']

        # LONG positions
        if pos['side'] == 'long':
            if tp is not None and current_price >= tp:
                # Only send reduce_only if amount > 0
                close_pos = place_market_order_ccxt(
                    side='sell',
                    amount=amount,
                    pos_side='LONG'
                )
                status = close_pos.get('status')
                print(f"[POS TP] with status = {status} pos={pos['id']} closed at {current_price:.2f} pos entry = {pos['entry']}")
                open_positions.remove(pos)
                has_open_position_zones.discard(grid_id)
                closed.append((pos, 'TP', current_price))
                
                #recreate_grid_order(pos['entry'], pos['side'])
                time.sleep(0.3)

            elif sl is not None and current_price <= sl:
                close_pos = place_market_order_ccxt(
                    side='sell',
                    amount=amount,
                    pos_side='LONG'
                )
                status = close_pos.get('status')
                print(f"[POS SL] with status = {status} pos={pos['id']} closed at {current_price:.2f} pos entry = {pos['entry']}")
                open_positions.remove(pos)
                has_open_position_zones.discard(grid_id)
                closed.append((pos, 'SL', current_price))
                # Mark that grid zone as blocked for LONG
                blocked_zones.append({
                    'grid_id': grid_id,
                    'price':pos['entry'],
                    'side': 'long',
                    'entry': entry_price,
                    'cooldown': True,
                    'sl_price': sl,
                    'timestamp': datetime.now(timezone.utc)
                })
                print(f"[BLOCKED] LONG zone blocked after SL at {sl:.2f} for grid={grid_id}")

                
                #recreate_grid_order(pos['entry'], pos['side'])
                time.sleep(0.3)

        # SHORT positions
        elif pos['side'] == 'short':
            if tp is not None and current_price <= tp:
                close_pos = place_market_order_ccxt(
                    side='buy',
                    amount=amount,
                    pos_side='SHORT'
                )
                status = close_pos.get('status')
                print(f"[POS TP] with status = {status} pos={pos['id']} closed at {current_price:.2f} pos entry = {pos['entry']}")
                open_positions.remove(pos)
                has_open_position_zones.discard(grid_id)
                closed.append((pos, 'TP_SHORT', current_price))
                
                #recreate_grid_order(pos['entry'], pos['side']
                time.sleep(0.3)
                
            elif sl is not None and current_price >= sl:
                close_pos = place_market_order_ccxt(
                    side='buy',
                    amount=amount,
                    pos_side='SHORT'
                )
                status = close_pos.get('status')
                print(f"[POS SL SHORT] with status = {status} pos={pos['id']} closed at {current_price:.2f} pos entry = {pos['entry']}")
                open_positions.remove(pos)
                has_open_position_zones.discard(grid_id)
                closed.append((pos, 'SL_SHORT', current_price))
                
                # Mark that grid zone as blocked for SHORT
                blocked_zones.append({
                    'grid_id': grid_id,
                    'price':pos['entry'],
                    'side': 'short',
                    'entry': entry_price,
                    'cooldown': True,
                    'sl_price': sl,
                    'timestamp': datetime.now(timezone.utc)
                })
                print(f"[BLOCKED] SHORT zone blocked after SL at {sl:.2f} for grid={grid_id}")

                
                #recreate_grid_order(pos['entry'], pos['side'])
                time.sleep(0.3)

    return closed


In [49]:
def try_update_order_statuses():
    """
    Poll ccxt for open orders and update statuses in open_orders.
    For simulated orders (simulate_only) we keep status as is until filled via price checks.
    """
    if simulate_only:
        return
    # fetch open orders for symbol
    try:
        live_open = exchange.fetch_open_orders(symbol)
        live_ids = {o['id'] for o in live_open}
        # mark orders that are no longer open as 'filled' or 'closed'
        for o in open_orders:
            ccid = o.get('ccxt_id')
            if not ccid:
                continue
            if ccid not in live_ids and o['status'] == 'open':
                # fetch order detail
                det = fetch_order_ccxt(ccid)
                if det:
                    status = det.get('status')
                    o['status'] = status
    except Exception as e:
        print("[CCXT POLL ERROR]", e)

In [50]:
  # Use set for faster lookup

def open_long(current_price, amount_usd):
    """Open LONG positions when the price drops between this grid and the grid below it."""
    global balance
    filled = []

    for o in list(open_orders):
        # Only check open BUY orders
        if o['status'] != 'open' or o['side'] != 'buy':
            continue
        
        # Skip if zone is blocked due to previous SL
        zone_blocked = any(
            b['grid_id'] == o['grid_id'] and b['side'] == 'long' and b['cooldown']
            for b in blocked_zones
        )
        if zone_blocked:
            # Still blocked — skip opening long here
            continue


        grid_id = o['grid_id']
        order_entry_price = o['price']
        
        if grid_id in has_open_position_zones:
            continue

        # Collect all buy prices for this grid
        grid_orders = [oo for oo in open_orders if oo['grid_id'] == grid_id and oo['side'] == 'buy']
        grid_prices = sorted([oo['price'] for oo in grid_orders])

        # Find this order's index and its lower grid level
        i = grid_prices.index(o['price'])
        lower_grid_price = grid_prices[i - 1] if i > 0 else None

        # Skip if there’s already a position inside this zone
        zone_has_position = False
        for p in open_positions:
            if p['grid_id'] == grid_id:
                if lower_grid_price:
                    if lower_grid_price < p['entry'] <= o['price']:
                        zone_has_position = True
                        break
                else:
                    if p['entry'] <= o['price']:
                        zone_has_position = True
                        break
        
        if zone_has_position:
            continue

        # Price must fall into this buying zone
        in_zone = (
            (lower_grid_price or 0) < current_price <= o['price']
        )

        if in_zone:
            size_usd = min(amount_usd, balance)
            if size_usd <= 0:
                print("❌ Not enough balance to open LONG.")
                continue

            amount = size_usd / current_price

            # Send a real market BUY order
            ccxt_order = place_market_order_ccxt(side='buy', amount=amount, pos_side='LONG')

            # Fetch real fill price or fallback
            entry_price = ccxt_order.get('average', current_price)
            qty = ccxt_order.get('amount', amount)

            # Compute real Take Profit and Stop Loss
            tp_price = entry_price + user_grid_spacing
            sl_price = entry_price - 0.5 * user_grid_spacing
            status = ccxt_order.get('status')

            # Save the opened position
            pos = {
                'id': str(uuid.uuid4())[:10],
                'side': 'long',
                'entry': float(entry_price),
                'order_entry_price': float(order_entry_price),
                'ccxt_id': ccxt_order.get('id'),
                'qty': qty,
                'notional': size_usd,
                'grid_id': grid_id,
                'created_at': datetime.now(timezone.utc),
                'last_price': current_price,
                'active': True,
                'tp_price': float(tp_price),
                'sl_price': float(sl_price)
            }

            # Update local memory
            open_positions.append(pos)
            has_open_position_zones.add(grid_id)
            

            print(f"[LONG FILLED] with status = {status} order={o['id']} entry={entry_price:.2f} qty={qty:.6f} notional={size_usd:.2f} TP={tp_price:.2f} SL={sl_price:.2f}")
            filled.append(o)

    return filled


def open_short(current_price, amount_usd):
    """Open SHORT positions when the price rises between this grid and the grid above it."""
    global balance
    filled = []

    for o in list(open_orders):
        # Only check open SELL orders
        if o['status'] != 'open' or o['side'] != 'sell':
            continue
        
        zone_blocked = any(
        b['grid_id'] == o['grid_id'] and b['side'] == 'short' and b['cooldown']
        for b in blocked_zones
        )
        if zone_blocked:
            continue


        grid_id = o['grid_id']
        order_entry_price = o['price']
        
        if grid_id in has_open_position_zones:
            continue

        # Collect all sell prices for this grid
        grid_orders = [oo for oo in open_orders if oo['grid_id'] == grid_id and oo['side'] == 'sell']
        grid_prices = sorted([oo['price'] for oo in grid_orders])

        # Find this order's index and its upper grid level
        i = grid_prices.index(o['price'])
        upper_grid_price = grid_prices[i + 1] if i + 1 < len(grid_prices) else None

        # Skip if there’s already a position inside this zone
        zone_has_position = False
        for p in open_positions:
            if p['grid_id'] == grid_id:
                if upper_grid_price:
                    if o['price'] <= p['entry'] < upper_grid_price:
                        zone_has_position = True
                        break
                else:
                    if p['entry'] >= o['price']:
                        zone_has_position = True
                        break
                    
        if zone_has_position:
            continue

        # Price must rise into the shorting zone
        in_zone = (
            upper_grid_price > current_price >= o['price']
            if upper_grid_price else current_price >= o['price']
        )

        if in_zone:
            size_usd = min(amount_usd, balance)
            if size_usd <= 0:
                print("❌ Not enough balance to open SHORT.")
                continue

            # Calculate the trade amount (BTC, ETH, etc.)
            amount = size_usd / current_price

            # Send a real market SELL order
            ccxt_order = place_market_order_ccxt(side='sell', amount=amount, pos_side='SHORT')

            # Fetch real fill price or fallback
            entry_price = ccxt_order.get('average', current_price)
            qty = ccxt_order.get('amount', amount)

            # Compute real Take Profit and Stop Loss
            tp_price = entry_price - user_grid_spacing
            sl_price = entry_price + 0.5 * user_grid_spacing
            status = ccxt_order.get('status')

            # Save the opened position
            pos = {
                'id': str(uuid.uuid4())[:10],
                'side': 'short',
                'entry': float(entry_price),
                'order_entry_price': float(order_entry_price),
                'ccxt_id': ccxt_order.get('id'),
                'qty': qty,
                'notional': size_usd,
                'grid_id': grid_id,
                'created_at': datetime.now(timezone.utc),
                'last_price': current_price,
                'active': True,
                'tp_price': float(tp_price),
                'sl_price': float(sl_price)
            }

            # Update local memory
            open_positions.append(pos)
            has_open_position_zones.add(grid_id)
            

            print(f"[SHORT FILLED] with status = {status} order={o['id']} entry={entry_price:.2f} qty={qty:.6f} notional={size_usd:.2f} TP={tp_price:.2f} SL={sl_price:.2f}")
            filled.append(o)

    return filled



In [51]:
def cancel_order_ccxt(ccxt_order_id):
    try:
        exchange.cancel_order(ccxt_order_id, symbol)
        return True
    except Exception as e:
        print("[CCXT CANCEL ERROR]", e)
        return False   

In [52]:
# -------------------------
# Graceful transition & frozen cutoffs
# -------------------------
def begin_graceful_transition(new_prediction, center_price):
    active_gids = [gid for gid, meta in grids.items() if not meta.get('frozen', False)]
    for gid in active_gids:
        freeze_grid(gid)
    any_open_in_frozen = any([len(get_open_orders_for_grid(gid)) > 0 or any(p['grid_id']==gid for p in open_positions) for gid in active_gids])
    fraction = position_fraction * (reduced_fraction if any_open_in_frozen else 1.0)
    dir_map = {2: 'up', 0: 'down', 1: 'hold'}
    direction = dir_map.get(new_prediction, 'hold')
    new_gid = place_grid(direction, center_price, mode='trend', fraction=fraction)
    print(f"[TRANSITION] started new grid {new_gid} dir={direction} fraction={fraction:.3f}")
    return new_gid

def check_frozen_cutoffs(current_price):
    global balance
    for gid, meta in list(grids.items()):
        if not meta.get('frozen', False): continue
        frozen_at = meta.get('frozen_at', datetime.now(timezone.utc))
        age = (datetime.now(timezone.utc) - frozen_at).total_seconds()
        orders = get_open_orders_for_grid(gid)
        exposure = sum([o['size_usd'] for o in orders])
        if exposure <= 0 and not any(p['grid_id'] == gid for p in open_positions):
            print(f"[CLEANUP] frozen grid {gid} cleared (no exposure).")
            grids.pop(gid, None); continue
        if age > frozen_timeout_seconds:
            print(f"[FORCED CUTOFF] grid {gid} timed out ({age:.0f}s) → closing all positions at market and cancelling orders.")
            for o in list(open_orders):
                if o['grid_id'] == gid and o['status'] == 'open':
                    ccid = o.get('ccxt_id')
                    if ccid: cancel_order_ccxt(ccid)
                    o['status'] = 'cancelled'
                    open_orders.remove(o)
            for p in list(open_positions):
                if p['grid_id'] == gid:
                    exit_price = current_price
                    notional = p['notional']
                    if p['side'] == 'long':
                        close_pos = place_market_order_ccxt(side='sell', amount=p['qty'], pos_side='LONG')
                        status = close_pos.get('status') if close_pos else 'failed'
                    else:
                        close_pos = place_market_order_ccxt(side='buy', amount=p['qty'], pos_side='SHORT')
                        status = close_pos.get('status') if close_pos else 'failed'
                    
                    print(f"[FORCED CLOSE] pos={p['id']} grid={gid} side={p['side']} entry={p['entry']:.2f} exit={exit_price:.2f} status={status}")
                    open_positions.remove(p)
            grids.pop(gid, None)
            print(f"[GRID REMOVED] {gid} after forced cutoff.\n")

def close_all_positions(current_price):
    global open_positions
    # attempt to auto-detect mode
    hedge = detect_hedge_mode()
    for p in list(open_positions):
        exit_price = current_price
        side = 'sell' if p['side'] == 'long' else 'buy'
        pos_side = None
        if hedge:
            pos_side = 'LONG' if p['side'] == 'long' else 'SHORT'
        print(f"[CLOSE ATTEMPT] id={p['id']} side={p['side']} qty={p['qty']:.8f} hedge={hedge} pos_side={pos_side}")
        res = place_market_order_ccxt(side, p['qty'], pos_side=pos_side, symbol_local=p.get('symbol', symbol))
        print(f"[CLOSE RESULT] {res}")
        # remove if closed or 'closed' in status
        if res and res.get('status') in ('closed', 'ok', 'filled') or res.get('id'):
            try:
                open_positions.remove(p)
            except ValueError:
                pass
        else:
            # if failed, log and continue (do not remove)
            print(f"[WARN] Forced close failed for pos {p['id']}: {res.get('error')}")


In [53]:
def unblock_zones(current_price):
    """Unblock zones when reversal happens"""
    global blocked_zones
    for b in list(blocked_zones):
        side = b['side']
        entry = b['entry']
        sl = b['sl_price']

        if side == 'long':
            # Wait for price to move back above entry (reversal up)
            if current_price > entry:
                print(f"[UNBLOCK] LONG zone at {entry:.2f} is now active again.")
                blocked_zones.remove(b)

        elif side == 'short':
            # Wait for price to move back below entry (reversal down)
            if current_price < entry:
                print(f"[UNBLOCK] SHORT zone at {entry:.2f} is now active again.")
                blocked_zones.remove(b)


In [54]:
# Store initial balance
initial_balance = exchange.fetch_balance(params={'type': 'future'})['total']['USDT']

def compute_realistic_profit(symbol):
    info = exchange.fetch_balance(params={'type': 'future'})['info']
    wallet_balance = float(info['totalWalletBalance'])       # includes realized PnL
    unrealized = float(info.get('totalUnrealizedProfit', 0)) # safety fallback

    simulated_balance = wallet_balance + unrealized
    net_profit = simulated_balance - initial_balance
    return simulated_balance, net_profit

print("initial_balance =", initial_balance)




initial_balance = 4910.92680649


In [55]:
def update_blocked_zones(current_price):
    """
    Release blocked zones if price has moved enough from the blocked entry.
    Example:
      - For LONG: if price drops 2 grids below blocked zone, unblock the upper grid.
      - For SHORT: if price rises 2 grids above blocked zone, unblock the lower grid.
    """
    global blocked_zones
    to_unblock = []

    for b in blocked_zones:
        grid_id = b['grid_id']
        side = b['side']
        entry_price = b['price']

        # Calculate difference between current price and blocked entry
        price_diff = abs(current_price - entry_price)

        # How far price must move to release (2x spacing recommended)
        release_distance = 2 * user_grid_spacing

        if side == 'long':
            # If price dropped much lower, unblock upper zones
            if current_price <= entry_price - release_distance:
                to_unblock.append(b)

        elif side == 'short':
            # If price rose much higher, unblock lower zones
            if current_price >= entry_price + release_distance:
                to_unblock.append(b)

    # Remove all zones marked for unblocking
    for b in to_unblock:
        blocked_zones.remove(b)
        print(f"[UNBLOCKED] {b['side'].upper()} zone for grid={b['grid_id']} released at price {current_price:.2f}")

    # Optional: print status summary
    if to_unblock:
        print(f"[UNBLOCK SUMMARY] {len(to_unblock)} zones released at price={current_price:.2f}")


In [56]:
def update_trailing_sl(pos, current_price):
    """Move SL upward to lock in profit once price moves significantly in favor."""
    global user_grid_spacing

    if pos['side'] == 'long':
        profit_move = current_price - pos['entry']
        if profit_move > 0.5 * user_grid_spacing:  # moved halfway toward TP
            new_sl = max(pos['sl_price'], pos['entry'] + 0.125 * user_grid_spacing)
            if new_sl > pos['sl_price']:
                print(f"[LOCK PROFIT LONG] Moved SL from {pos['sl_price']:.2f} → {new_sl:.2f} for pos={pos['id']}")
                pos['sl_price'] = new_sl

    elif pos['side'] == 'short':
        profit_move = pos['entry'] - current_price
        if profit_move > 0.5 * user_grid_spacing:  # moved halfway toward TP
            new_sl = min(pos['sl_price'], pos['entry'] - 0.125 * user_grid_spacing)
            if new_sl < pos['sl_price']:
                print(f"[LOCK PROFIT SHORT] Moved SL from {pos['sl_price']:.2f} → {new_sl:.2f} for pos={pos['id']}")
                pos['sl_price'] = new_sl


In [57]:
def update_trailing_sl_2(pos, current_price):
    grid_spacing = user_grid_spacing
    move_threshold = 0.0625 * grid_spacing  # move SL every 0.25 grid
    
    entry_price = pos['entry']
    sl_price = pos['sl_price']
    side = pos['side']

    # LONG position logic
    if side == 'long':
        profit_distance = current_price - entry_price
        if profit_distance >= move_threshold:
            # how many steps of 0.25 grid have we moved?
            steps = int(profit_distance // move_threshold)
            new_sl = entry_price - grid_spacing + (steps * move_threshold)

            if new_sl > sl_price:
                pos['sl_price'] = new_sl
                print(f"[TRAIL LONG] SL moved up to {new_sl:.2f} (steps={steps})")

    # SHORT position logic
    elif side == 'short':
        profit_distance = entry_price - current_price
        if profit_distance >= move_threshold:
            steps = int(profit_distance // move_threshold)
            new_sl = entry_price + grid_spacing - (steps * move_threshold)

            if new_sl < sl_price:
                pos['sl_price'] = new_sl
                print(f"[TRAIL SHORT] SL moved down to {new_sl:.2f} (steps={steps})")


In [ ]:
# -------------------------
# Main run loop (refactored)
# -------------------------

def run_bot():
    wait_for_reversal = False
    global balance, open_positions, open_orders, grids
    last_prediction = None
    last_prediction_time = None
    last_trend_direction = None
    
    try:
        balance_data = exchange.fetch_balance()
        balance = float(balance_data['total'].get('USDT', 0))
    except Exception as e:
        print("[INIT BALANCE ERROR]", e)
        balance = 0.0

    # Warm-up: compute last reversal
    back_df = fetch_latest_ohlcv(symbol, model_timeframe, 120 + window_size)
    feat_df = compute_indicators(back_df).dropna()

    preds = [
        predict_label_from_window(feat_df[FEATURES].values[i - window_size:i])
        for i in range(window_size, len(feat_df))
    ]
    last_reversal = next((p for p in reversed(preds) if p != 1), None)
    #last_prediction = last_reversal if last_reversal is not None else 1
    last_prediction = 2
    last_prediction_time = datetime.now(timezone.utc)

    try:
        last_price = float(exchange.fetch_ticker(symbol)['last'])
    except Exception:
        # fallback
        last_price = float(fetch_latest_ohlcv_2(symbol, price_timeframe, 2)['close'].values[-1])
    print(f"[INIT] last_pred={last_prediction} ({'DOWN' if last_prediction==0 else 'HOLD' if last_prediction==1 else 'UP'}), price={last_price:.2f}")

    # Place initial grid based on last reversal
    if last_prediction == 0:
        place_grid('down', last_price, mode='trend')
        last_trend_direction = 'short'
    elif last_prediction == 2:
        
        print(f"[INIT STATUS] Starting grid trading in {'up/hold'.upper()} mode.")
        place_grid('up', last_price, mode='trend')
        last_trend_direction = 'long' if last_prediction in (1, 2) else 'short'
        
    else:
        print("Waiting for clear trend signal to start grid...")    

    try:
        while True:
            print("initial_balance =", initial_balance)
            now = datetime.now(timezone.utc)
            do_predict = (last_prediction_time is None) or ((now - last_prediction_time).total_seconds() >= prediction_interval)
            
            try:
                balance_data = exchange.fetch_balance()
                balance = float(balance_data['total'].get('USDT', 0))
            except Exception as e:
                print("[BALANCE FETCH ERROR]", e)


            if do_predict:
                # --- Model prediction ---
                df_feat = fetch_latest_ohlcv(symbol, model_timeframe, limit)
                feat_df = compute_indicators(df_feat).dropna()
                if len(feat_df) >= window_size:
                    window = feat_df[FEATURES].values[-window_size:]
                    new_pred = predict_label_from_window(window)
                    print(f"[{datetime.now(timezone.utc)}] ML pred={new_pred}")

                    if new_pred != last_prediction:
                        center_price = float(fetch_latest_ohlcv_2(symbol, price_timeframe, 2)['close'].values[-1])

                        # DOWN prediction → cancel everything, start short-grid
                        if new_pred == 0:
                            print("[⚠️ PREDICTION DOWN] -> Closing ALL positions, starting SHORT grid.")
                            close_all_positions(center_price)
                            for gid in list(grids.keys()):
                                cancel_grid_orders_ccxt(gid)
                                grids.pop(gid, None)
                            place_grid('down', center_price, mode='trend')
                            last_prediction = 0
                            last_trend_direction = 'short'

                        # UP or HOLD → cancel short-grid if any, start/maintain long-grid
                        elif new_pred == 2:
                            if last_prediction == 0:
                                print("[RESUME] Model predicts HOLD/UP -> Resuming long-grid.")
                                close_all_positions(center_price)
                                for gid in list(grids.keys()):
                                    cancel_grid_orders_ccxt(gid)
                                    grids.pop(gid, None)
                            last_prediction = new_pred
                            place_grid('up', center_price, mode='trend')
                            last_trend_direction = 'long'
                            
                        elif new_pred == 1 and last_trend_direction == 'short':
                            print("[⚠️ PREDICTION STILL DOWN] -> Closing ALL positions, CONTINUE SHORT grid.")
                            close_all_positions(center_price)
                            for gid in list(grids.keys()):
                                cancel_grid_orders_ccxt(gid)
                                grids.pop(gid, None)
                            place_grid('down', center_price, mode='trend')
                            last_prediction = 1
                            last_trend_direction = 'short'
                            
                        elif new_pred == 1 and last_trend_direction == 'long':
                            print("[⚠️ PREDICTION STILL UP] -> Closing ALL positions, CONTINUE LONG grid.")
                            close_all_positions(center_price)
                            for gid in list(grids.keys()):
                                cancel_grid_orders_ccxt(gid)
                                grids.pop(gid, None)
                            place_grid('up', center_price, mode='trend')
                            last_prediction = 1
                            last_trend_direction = 'long'    
                            
                    else:
                        print("[PREDICT] unchanged")
                last_prediction_time = datetime.now(timezone.utc)

            # --- Inner loop until next prediction ---
            inner_deadline = datetime.now(timezone.utc) + timedelta(seconds=prediction_interval)
            while datetime.now(timezone.utc) < inner_deadline:
                # --- Fetch latest price & indicators ---
                price_df = fetch_latest_ohlcv_2(symbol, price_timeframe, 2)
                try:
                    last_price = float(exchange.fetch_ticker(symbol)['last'])
                except Exception:
                    # fallback
                    last_price = float(fetch_latest_ohlcv_2(symbol, price_timeframe, 2)['close'].values[-1])
                feat_df = compute_indicators(fetch_latest_ohlcv(symbol, model_timeframe, limit)).dropna()
                if len(feat_df) >= window_size:
                    window = feat_df[FEATURES].values[-window_size:]
                    current_pred = predict_label_from_window(window)
                else:
                    current_pred = last_prediction

                

                # --- WAIT MODE: skip trading but monitor for reversal ---
                if wait_for_reversal:
                    if (current_pred == 0 and last_trend_direction != 'short') or \
                    (current_pred == 2 and last_trend_direction != 'long'):
                        print("[✅ REVERSAL DETECTED] Exiting WAIT MODE, resuming trading.")
                        wait_for_reversal = False

                        # Reset grids before resuming
                        for gid in list(grids.keys()):
                            cancel_grid_orders_ccxt(gid)
                            grids.pop(gid, None)

                        if current_pred == 0:
                            place_grid('down', last_price, mode='trend')
                            last_trend_direction = 'short'
                        elif current_pred == 2:
                            place_grid('up', last_price, mode='trend')
                            last_trend_direction = 'long'

                        last_prediction = current_pred
                    else:
                        print(f"[WAIT MODE ACTIVE] bal={balance:.2f}, profit={profit_now:.2f} — waiting for reversal")
                        time.sleep(check_interval_seconds)
                        continue

                # -------------------------
                # Handle dynamic switching
                # -------------------------
                if current_pred == 0 and last_trend_direction == 'long':
                    print("[⚠️ INNER LOOP: DOWN] Switching to SHORT grid")
                    close_all_positions(last_price)
                    for gid in list(grids.keys()):
                        cancel_grid_orders_ccxt(gid)
                        grids.pop(gid, None)
                    place_grid('down', last_price, mode='trend')
                    last_prediction = 0
                    last_trend_direction = 'short'

                elif current_pred == 2 and last_trend_direction == 'short':
                    print("[⬆️ INNER LOOP: UP] Switching to LONG grid")
                    close_all_positions(last_price)
                    for gid in list(grids.keys()):
                        cancel_grid_orders_ccxt(gid)
                        grids.pop(gid, None)
                    place_grid('up', last_price, mode='trend')
                    last_prediction = 2
                    last_trend_direction = 'long'

                
                clossed = try_resolve_positions_by_tp_sl(last_price)
                
                print(f"pos = {len(open_positions)}, closed = {len(clossed)}")

                # --- Trading activity ---
                trade_usd = max(20.0, balance * position_fraction) if balance > 0 else 0
                if last_prediction == 2:
                    open_long(last_price, trade_usd)
                elif last_prediction == 0:
                    open_short(last_price, trade_usd)
                elif current_pred == 1 and last_trend_direction == 'long':
                    open_long(last_price, trade_usd)
                elif current_pred == 1 and last_trend_direction == 'short':
                    open_short(last_price, trade_usd)
                    
                print(f"pos after open = {len(open_positions)}")    

                check_frozen_cutoffs(last_price)

                # --- Grid cleanup ---
                for gid, meta in list(grids.items()):
                    if not meta.get('frozen', False):
                        orders = get_open_orders_for_grid(gid)
                        has_pos = any(p['grid_id'] == gid for p in open_positions)
                        if len(orders) == 0 and not has_pos:
                            print(f"[CLEANUP] active grid {gid} cleared (no exposure).")
                            grids.pop(gid, None)

                # --- Safety exposure & place missing grids ---
                desired_dir = None
                exposure = total_open_exposure()
                exposure_pct = exposure / max(1e-8, balance)
                allow_place = exposure_pct <= max_total_exposure
                active_dirs = [meta['direction'] for gid, meta in grids.items() if not meta.get('frozen', False)]

                if last_prediction == 2:
                    desired_dir = 'up'
                elif last_prediction == 0:
                    desired_dir = 'down'
                elif current_pred == 1 and last_trend_direction == 'long':
                    desired_dir = 'up'
                elif current_pred == 1 and last_trend_direction == 'short':
                    desired_dir = 'down'

                if desired_dir not in active_dirs and allow_place:
                    place_grid(desired_dir, last_price, mode='trend')
                    
                simulated_balance, profit_now = compute_realistic_profit(symbol)
                
                # --- Drawdown protection ---
                if profit_now <= -6.20:
                    print("[❌ MAX DRAWDOWN] Closing all positions, entering WAIT MODE.")
                    close_all_positions(last_price)
                    for gid in list(grids.keys()):
                        cancel_grid_orders_ccxt(gid)
                        grids.pop(gid, None)
                    wait_for_reversal = True
                    break  # exit inner loop to outer loop

                # --- Debug / Status printing ---
                print(f"[STATUS] balance={balance:.2f} (sim if stopped now: {simulated_balance:.2f}, profit={profit_now:.2f}), "
                    f"exposure={exposure:.2f} ({exposure_pct*100:.1f}%), grids={len(grids)}")
                print(f"[{datetime.now(timezone.utc)}] price={last_price:.2f} pred={last_prediction} bal={balance:.2f} "
                    f"pos={len(open_positions)} open_orders={len([o for o in open_orders if o['status']=='open'])} grids={len(grids)}")
                print("------------------------------------------------------------")

                time.sleep(check_interval_seconds)


    except KeyboardInterrupt:
        print("Stopped by user. Balance:", balance)
        print("Open positions:", open_positions)
        print("Open orders:", [o for o in open_orders if o['status']=='open'])

# -------------------------
# Run
# -------------------------
if __name__ == "__main__":
    run_bot()

'# -------------------------\n# Main run loop (refactored)\n# -------------------------\n\ndef run_bot():\n    wait_for_reversal = False\n    global balance, open_positions, open_orders, grids\n    last_prediction = None\n    last_prediction_time = None\n    last_trend_direction = None\n\n    try:\n        balance_data = exchange.fetch_balance()\n        balance = float(balance_data[\'total\'].get(\'USDT\', 0))\n    except Exception as e:\n        print("[INIT BALANCE ERROR]", e)\n        balance = 0.0\n\n    # Warm-up: compute last reversal\n    back_df = fetch_latest_ohlcv(symbol, model_timeframe, 120 + window_size)\n    feat_df = compute_indicators(back_df).dropna()\n\n    preds = [\n        predict_label_from_window(feat_df[FEATURES].values[i - window_size:i])\n        for i in range(window_size, len(feat_df))\n    ]\n    last_reversal = next((p for p in reversed(preds) if p != 1), None)\n    #last_prediction = last_reversal if last_reversal is not None else 1\n    last_prediction 

In [59]:
open_orders.clear()
open_positions.clear()
grids.clear()


In [60]:
wait_for_reversal = False
global balance, open_positions, open_orders, grids
last_prediction = None
last_prediction_time = None
last_trend_direction = None


# Warm-up: compute last reversal
back_df = fetch_latest_ohlcv(symbol, model_timeframe, 120 + window_size)
feat_df = compute_indicators(back_df).dropna()

preds = [
predict_label_from_window(feat_df[FEATURES].values[i - window_size:i])
for i in range(window_size, len(feat_df))
]
last_reversal = next((p for p in reversed(preds) if p != 1), None)
last_prediction = last_reversal if last_reversal is not None else 1
print(f"[INIT] last_pred={last_prediction} ({'DOWN' if last_prediction==0 else 'HOLD' if last_prediction==1 else 'UP'})")

last_price = float(exchange.fetch_ticker(symbol)['last'])
user_grid_spacing =  200              #get_current_atr() * atr_multiplier
print(f"[GRID SPACING] initial ATR-based spacing = {user_grid_spacing:.2f}")

if last_prediction == 0:
    place_grid('down', last_price, mode='trend')
    last_trend_direction = 'short'
elif last_prediction == 2:
    place_grid('up', last_price, mode='trend')
    last_trend_direction = 'long'
else:
    print("Waiting for clear trend signal to start grid...")  
    
for order in open_orders:
    print(f"Order ID: {order['id']}, Price: {order['price']}")
  




[INIT] last_pred=2 (UP)
[GRID SPACING] initial ATR-based spacing = 200.00
[GRID PLACED] id=165c9624, dir=up, mode=trend, levels=17
------------------------------------------------------------
[DEBUG] Grid Details for 165c9624:
Grid 1/17 | Price: 113866.70 | TP: 114066.70 with difference = 200.0 | SL: 113766.70 with difference = -100.0
Grid 2/17 | Price: 114066.70 | TP: 114266.70 with difference = 200.0 | SL: 113966.70 with difference = -100.0
Grid 3/17 | Price: 114266.70 | TP: 114466.70 with difference = 200.0 | SL: 114166.70 with difference = -100.0
Grid 4/17 | Price: 114466.70 | TP: 114666.70 with difference = 200.0 | SL: 114366.70 with difference = -100.0
Grid 5/17 | Price: 114666.70 | TP: 114866.70 with difference = 200.0 | SL: 114566.70 with difference = -100.0
Grid 6/17 | Price: 114866.70 | TP: 115066.70 with difference = 200.0 | SL: 114766.70 with difference = -100.0
Grid 7/17 | Price: 115066.70 | TP: 115266.70 with difference = 200.0 | SL: 114966.70 with difference = -100.0
Gri

/tmp/ipykernel_6680/1118950588.py:16: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  'created_at': datetime.utcnow(),


In [ ]:
profit_tracker = {}  # {pos_id: {'peak': float, 'last_profit': float}}

profit_drop_threshold = 0.5   # USDT drop from peak to trigger close
min_profit_to_activate = 1.0  # start trailing once this profit reached
fee_per_trade = 0.05 / 100    # 0.05% taker fee

def get_position_pnl(pos, current_price):
    """Return approximate PnL in USDT for a given position."""
    entry = pos['entry']
    qty = pos['qty']
    notional = pos['notional']
    side = pos['side']
    
    # Price difference in % of entry
    if side == 'long':
        pnl = (current_price - entry) * qty
    else:  # short
        pnl = (entry - current_price) * qty

    # Subtract taker fees (entry + exit)
    pnl_after_fee = pnl - (notional * fee_per_trade * 2)
    return pnl_after_fee


In [ ]:
def update_and_check_profit_trailing(current_price):
    """Track profit and close positions if profit drops by threshold."""
    global open_positions

    for pos in list(open_positions):
        pos_id = pos['id']
        current_profit = get_position_pnl(pos, current_price)

        # Initialize tracking record
        if pos_id not in profit_tracker:
            profit_tracker[pos_id] = {'peak': current_profit, 'last_profit': current_profit}
        record = profit_tracker[pos_id]

        # Update peak profit
        if current_profit > record['peak']:
            record['peak'] = current_profit

        # --- Logic 1: stop early if immediate loss beyond threshold (including fee)
        if current_profit <= -profit_drop_threshold:
            close_pos = place_market_order_ccxt(
                side='sell' if pos['side'] == 'long' else 'buy',
                amount=pos['qty'],
                pos_side=pos['side'].upper()
            )
            print(f"[AUTO STOP LOSS] pos={pos_id} closed early at {current_price:.2f} profit={current_profit:.2f}")
            open_positions.remove(pos)
            continue

        # --- Logic 2: lock profit drop from peak ---
        peak_profit = record['peak']
        drawdown = peak_profit - current_profit

        if peak_profit >= min_profit_to_activate and drawdown >= profit_drop_threshold:
            close_pos = place_market_order_ccxt(
                side='sell' if pos['side'] == 'long' else 'buy',
                amount=pos['qty'],
                pos_side=pos['side'].upper()
            )
            print(f"[PROFIT TRAIL EXIT] pos={pos_id} peak={peak_profit:.2f} current={current_profit:.2f} → closed")
            open_positions.remove(pos)
            continue


In [ ]:
"""while True:
    
    print(f"[GRID SPACING] current ATR-based spacing = {user_grid_spacing:.2f}")
    
        
    current_pred = last_prediction
    trade_usd = 100
    last_price = float(exchange.fetch_ticker(symbol)['last'])
    
    for pos in open_positions:
        update_trailing_sl_2(pos, last_price)
    
    for pos in open_positions:
        update_trailing_sl(pos, last_price)

    
    closed = try_resolve_positions_by_tp_sl(last_price)
    unblock_zones(last_price)
    update_blocked_zones(last_price)
    if current_pred == 0:
        open_short(last_price, trade_usd)
        last_direction = 'short'
    elif current_pred == 2:
        open_long(last_price, trade_usd)
        last_direction = 'long'
    
    for pos in open_positions:
        print(f"Open Position ID: {pos['id']}, Side: {pos['side']}, Entry: {pos['entry']}, Qty: {pos['qty']}")         
    print(f"pos = {len(open_positions)}, closed = {len(closed)}")  
    
            
    time.sleep(1)
        
        """

[GRID SPACING] current ATR-based spacing = 200.00
[LONG FILLED] with status = closed order=30e13c4c-c entry=115543.80 qty=0.001000 notional=100.00 TP=115743.80 SL=115443.80
Open Position ID: 09911026-7, Side: long, Entry: 115543.8, Qty: 0.001
pos = 1, closed = 0
[GRID SPACING] current ATR-based spacing = 200.00
Open Position ID: 09911026-7, Side: long, Entry: 115543.8, Qty: 0.001
pos = 1, closed = 0
[GRID SPACING] current ATR-based spacing = 200.00
Open Position ID: 09911026-7, Side: long, Entry: 115543.8, Qty: 0.001
pos = 1, closed = 0
[GRID SPACING] current ATR-based spacing = 200.00
Open Position ID: 09911026-7, Side: long, Entry: 115543.8, Qty: 0.001
pos = 1, closed = 0
[GRID SPACING] current ATR-based spacing = 200.00
Open Position ID: 09911026-7, Side: long, Entry: 115543.8, Qty: 0.001
pos = 1, closed = 0
[GRID SPACING] current ATR-based spacing = 200.00
Open Position ID: 09911026-7, Side: long, Entry: 115543.8, Qty: 0.001
pos = 1, closed = 0
[GRID SPACING] current ATR-based spac

In [ ]:
def zone_has_position_for_order(grid_id, level_price, tol=1e-3):
    for p in open_positions:
        if p.get('grid_id') != grid_id: 
            continue
        if not p.get('active', True): 
            continue
        # consider same zone if entry is within half spacing or small tolerance
        if abs(p['entry'] - level_price) <= tol or abs(p['entry'] - level_price) < 0.5 * user_grid_spacing:
            return True
    return False


In [ ]:
for o in open_orders:
    grid_id = o['grid_id']
    # Collect all sell prices for this grid
    grid_orders = [oo for oo in open_orders if oo['grid_id'] == grid_id and oo['side'] == 'sell']
    grid_prices = sorted([oo['price'] for oo in grid_orders])

    # Find this order's index and its upper grid level
    i = grid_prices.index(o['price'])
    upper_grid_price = grid_prices[i + 1] if i + 1 < len(grid_prices) else None
    zone_has_position = any(
            p['grid_id'] == grid_id and o['price'] <= p['entry'] < (upper_grid_price or float('inf'))
            for p in open_positions
        )
    print(o['price'], zone_has_position)

110693.4 False
110743.4 False
110793.4 False
110843.4 False
110893.4 False
110943.4 False
110993.4 False
111043.4 False
111093.4 False
111143.4 False
111193.4 False
111243.4 False
111293.4 False
111343.4 False
111393.4 False
111443.4 False
111493.4 False


In [ ]:
for p in blocked_zones:
    print(p)

{'grid_id': 'c8ff7993', 'side': 'long', 'entry': 111374.9, 'cooldown': True, 'sl_price': 111349.9, 'timestamp': datetime.datetime(2025, 10, 24, 12, 48, 29, 581069, tzinfo=datetime.timezone.utc)}
